# ตัวอย่างตัวแทนจองโรงแรมและเที่ยวบิน

โซลูชันนี้จะช่วยคุณจองตั๋วเครื่องบินและโรงแรม สถานการณ์คือทริปจากลอนดอนฮีทโธรว์ LHR วันที่ 20 กุมภาพันธ์ 2024 ไปนครนิวยอร์ก JFK กลับวันที่ 27 กุมภาพันธ์ 2025 บินชั้นประหยัดกับ British Airways เท่านั้น ฉันต้องการเข้าพักที่โรงแรมฮิลตันในนครนิวยอร์ก กรุณาให้ราคาสำหรับเที่ยวบินและโรงแรมด้วย


# เริ่มต้นบริการ Azure AI Agent และรับข้อมูลการกำหนดค่าจาก **.env**

### **.env**

สร้างไฟล์ .env

**.env** ประกอบด้วยสตริงการเชื่อมต่อของบริการ Azure AI Agent, โมเดลที่ใช้โดย AOAI และ API บริการค้นหาของ Google ที่สอดคล้องกัน, ENDPOINT เป็นต้น

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "ชื่อการปรับใช้โมเดลบริการ Azure AI Agent ของคุณ"

[**NOTE**] คุณจะต้องมีโมเดลพร้อม Rate Limit 100,000 (tokens ต่อ นาที) และ Rate Limit 600 (คำขอต่อ นาที)

  คุณสามารถรับโมเดลได้จาก Microsoft Foundry - Model and Endpoint

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "สตริงการเชื่อมต่อโครงการบริการ Azure AI Agent ของคุณ"

  คุณสามารถรับสตริงการเชื่อมต่อโครงการได้ในภาพรวมโครงการของคุณในหน้าจอ AI ​​Foundry Portal

- **SERPAPI_SEARCH_API_KEY** = "คีย์ API ค้นหา SERPAPI ของคุณ"
- **SERPAPI_SEARCH_ENDPOINT** = "จุดสิ้นสุดการค้นหา SERPAPI ของคุณ"

ในการรับชื่อการปรับใช้โมเดลและสตริงการเชื่อมต่อโครงการของบริการ Azure AI Agent คุณจำเป็นต้องสร้างบริการ Azure AI Agent แนะนำให้ใช้ [แม่แบบนี้](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) เพื่อสร้างโดยตรง （***หมายเหตุ:*** ปัจจุบันบริการ Azure AI Agent ถูกตั้งค่าในภูมิภาคจำกัด แนะนำให้คุณดูที่ [ลิงก์นี้](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) เพื่อกำหนดภูมิภาค)

Agent จำเป็นต้องเข้าถึง SERPAPI แนะนำให้ลงทะเบียนโดยใช้ [ลิงก์นี้](https://serpapi.com/searches) หลังจากลงทะเบียนแล้ว คุณจะได้รับคีย์ API ที่ไม่ซ้ำและ ENDPOINT


# Setup 

ในการรันโน้ตบุ๊กนี้ คุณจะต้องตรวจสอบให้แน่ใจว่าคุณได้ติดตั้งไลบรารีที่จำเป็นโดยการรันคำสั่ง `pip install -r requirements.txt` แล้ว


In [ ]:
from semantic_kernel import __version__

__version__

เวอร์ชัน Semantic Kernel ของคุณควรเป็นอย่างน้อย 1.27.2


โหลดการตั้งค่าและทรัพยากรจากไฟล์ .env ของคุณ โปรดตรวจสอบให้แน่ใจว่าคุณได้เพิ่มคีย์และการตั้งค่าของคุณแล้ว และสร้างไฟล์ .env ในเครื่องของคุณเรียบร้อยแล้ว


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# ลงชื่อเข้าใช้ Azure

ตอนนี้คุณต้องลงชื่อเข้าใช้ Azure เปิดเทอร์มินัลแล้วรันคำสั่งต่อไปนี้:

```bash
az login
```

คำสั่งนี้จะแจ้งให้คุณป้อนข้อมูลรับรอง Azure ของคุณ เพื่อให้บริการ Azure AI Agent ทำงานได้อย่างถูกต้อง


# Explanation:
นี่คือค่าตัวแปรที่เก็บ API key สำหรับการเข้าถึงบริการ API ของ SERP (Search Engine Results Page) API key คือรหัสเฉพาะที่ใช้สำหรับยืนยันตัวตนของคำขอที่เชื่อมโยงกับบัญชีของคุณ

Purpose: จุดประสงค์ของบรรทัดนี้คือการเก็บ API key ไว้ในตัวแปรเพื่อใช้ในการยืนยันตัวตนของคำขอไปยังบริการ SERP API ซึ่งจำเป็นต้องใช้ API key ในการเข้าถึงบริการและทำการค้นหา
How to Get a SERP API Key: ในการรับ API key ของ SERP ให้ทำตามขั้นตอนทั่วไปที่ https://serpapi.com (ขั้นตอนที่แน่นอนอาจแตกต่างกันขึ้นอยู่กับบริการ SERP API ที่คุณใช้):

Choose a SERP API Service: มีบริการ SERP API หลายตัว เช่น SerpAPI, Google Custom Search JSON API และอื่นๆ เลือกบริการที่เหมาะสมกับความต้องการของคุณมากที่สุด

Sign Up for an Account: ไปที่เว็บไซต์ของบริการ SERP API ที่เลือกและสมัครบัญชี คุณอาจต้องระบุข้อมูลพื้นฐานบางอย่างและยืนยันอีเมลของคุณ

Create an API Key: หลังจากสมัครแล้ว ให้เข้าสู่ระบบบัญชีของคุณและไปที่ส่วน API หรือแดชบอร์ด ค้นหาตัวเลือกเพื่อสร้างหรือสร้าง API key ใหม่
Copy the API Key to your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: นี่คือแปรที่เก็บ URL พื้นฐานสำหรับจุดสิ้นสุดของ SERP API ชื่อแปร BASE_URL เป็น convention ที่ใช้เพื่อบ่งบอกว่า URL นี้เป็นจุดเริ่มต้นสำหรับการทำคำขอ API
'https://serpapi.com/search':

นี่คือสตริง URL จริงที่ถูกกำหนดให้กับแปร BASE_URL ซึ่งแสดงถึงจุดสิ้นสุดสำหรับการทำคำค้นหาผ่าน SERP API

# Purpose:
วัตถุประสงค์ของบรรทัดนี้คือการกำหนดค่าคงที่ที่เก็บ URL พื้นฐานสำหรับ SERP API URL นี้จะถูกใช้เป็นจุดเริ่มต้นสำหรับการสร้างคำขอ API เพื่อดำเนินการค้นหา

# Usage:
โดยการกำหนด URL พื้นฐานลงในแปร คุณสามารถนำมาใช้ซ้ำได้อย่างง่ายดายในโค้ดของคุณทุกครั้งที่ต้องทำคำขอไปยัง SERP API ซึ่งทำให้โค้ดของคุณดูแลได้ง่ายขึ้นและลดความเสี่ยงของข้อผิดพลาดจากการเขียน URL ซ้ำในหลายๆ ที่ ตัวอย่างปัจจุบันคือ https://serpapi.com/search?engine=bing ซึ่งใช้ Bing search API สามารถเลือกใช้ API ต่างๆ ได้ที่ https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# คำอธิบาย:

นี่คือที่ที่โค้ดปลั๊กอินของคุณถูกเก็บไว้

การกำหนดคลาส: `class BookingPlugin`: กำหนดคลาสชื่อ BookingPlugin ซึ่งมีเมธอดสำหรับการจองโรงแรมและเที่ยวบิน

เมธอดการจองโรงแรม:

- `@kernel_function(description="booking hotel")`: ตัวตกแต่งที่อธิบายฟังก์ชันเป็นฟังก์ชันเคอร์เนลสำหรับการจองโรงแรม
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: กำหนดเมธอดสำหรับการจองโรงแรมพร้อมพารามิเตอร์และประเภทการคืนค่าที่ระบุไว้

เมธอดนี้จะสร้างพจนานุกรมของพารามิเตอร์สำหรับคำขอจองโรงแรมและส่งคำขอ GET ไปยัง SERP API ตรวจสอบสถานะการตอบกลับและคืนค่าคุณสมบัติของโรงแรมถ้าสำเร็จ หรือคืนค่า None ถ้าคำขอไม่สำเร็จ

เมธอดการจองเที่ยวบิน:

- `@kernel_function(description="booking flight")`: ตัวตกแต่งที่อธิบายฟังก์ชันเป็นฟังก์ชันเคอร์เนลสำหรับการจองเที่ยวบิน
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: กำหนดเมธอดสำหรับการจองเที่ยวบินพร้อมพารามิเตอร์และประเภทการคืนค่าที่ระบุไว้

เมธอดนี้จะสร้างพจนานุกรมของพารามิเตอร์สำหรับคำขอเที่ยวบินขาออกและขากลับ และส่งคำขอ GET ไปยัง SERP API ตรวจสอบสถานะการตอบกลับและต่อท้ายข้อมูลเที่ยวบินลงในสตริงผลลัพธ์ถ้าสำเร็จ หรือพิมพ์ข้อความแสดงข้อผิดพลาดถ้าคำขอไม่สำเร็จ เมธอดนี้คืนค่าสตริงผลลัพธ์ที่มีข้อมูลเที่ยวบิน


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# คำอธิบาย:
คำสั่งนำเข้า: นำเข้าโมดูลที่จำเป็นสำหรับใบรับรองของ Azure, ตัวแทน AI, เนื้อหาข้อความแชท, บทบาทผู้เขียน และตัวตกแต่งฟังก์ชันเคอร์เนล

ตัวจัดการบริบทแบบอะซิงโครนัส: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): ตั้งค่าตัวจัดการบริบทแบบอะซิงโครนัสเพื่อจัดการกับใบรับรอง Azure และสร้างไคลเอนต์ตัวแทน AI

ชื่อและคำแนะนำของตัวแทน: 
- `AGENT_NAME = "BookingAgent"`: กำหนดชื่อของตัวแทน
- `AGENT_INSTRUCTIONS = """..."""`: ให้คำแนะนำอย่างละเอียดสำหรับตัวแทนเกี่ยวกับวิธีจัดการคำขอจอง

สร้างคำนิยามตัวแทน: `agent_definition = await client.agents.create_agent(...)`: สร้างคำนิยามตัวแทนด้วยโมเดล ชื่อ และคำแนะนำที่ระบุ

สร้างตัวแทน AzureAI: `agent = AzureAIAgent(...)`: สร้างตัวแทน AzureAI โดยใช้ไคลเอนต์ คำนิยามตัวแทน และปลั๊กอินที่กำหนด

สร้างเธรด: `thread: AzureAIAgentThread | None = None`: สร้างเธรดสำหรับตัวแทน ไม่จำเป็นต้องสร้างเธรดก่อน - หากระบุค่าเป็น `None` เธรดใหม่จะถูกสร้างขึ้นในระหว่างการเรียกใช้งานครั้งแรกและส่งกลับเป็นส่วนหนึ่งของการตอบกลับ

อินพุตของผู้ใช้: `user_inputs = ["..."]`: กำหนดรายการอินพุตของผู้ใช้ให้ตัวแทนประมวลผล

ในบล็อก finally ให้ลบเธรดและตัวแทนเพื่อทำความสะอาดทรัพยากร


# Authentication

คลาส `DefaultAzureCredential` เป็นส่วนหนึ่งของ Azure SDK สำหรับ Python ซึ่งให้วิธีการเริ่มต้นในการยืนยันตัวตนกับบริการของ Azure โดยพยายามยืนยันตัวตนโดยใช้วิธีการหลายวิธีในลำดับที่กำหนด เช่น ตัวแปรสภาพแวดล้อม, managed identity, และข้อมูลประจำตัวของ Azure CLI

การทำงานแบบอะซิงโครนัส: โมดูล aio บ่งชี้ว่าคลาส DefaultAzureCredential รองรับการทำงานแบบอะซิงโครนัส ซึ่งหมายความว่าคุณสามารถใช้มันกับ asyncio เพื่อดำเนินการร้องขอยืนยันตัวตนแบบไม่บล็อกได้


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**คำปฏิเสธความรับผิดชอบ**: เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) ถึงแม้เราจะพยายามให้ความถูกต้อง โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความคลาดเคลื่อน เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลสำคัญแนะนำให้ใช้การแปลโดยผู้เชี่ยวชาญมนุษย์ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดขึ้นจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
